In [100]:
#Imports
import pandas as pd
import numpy as np
from zipfile import ZipFile
import requests
from io import StringIO, BytesIO
import os

from datetime import date,datetime,timedelta

import geopandas as gpd
from shapely.geometry import shape 
import shapefile
import osr
import bokeh
import json

In [116]:
import scraper
tables = scraper.get_tables()
cases = pd.DataFrame({key:value['Confirmed'] for (key,value) in tables.items() if key not in ['Active','Tests','Quebec, Canada']})
cases.index.name = 'Date'

In [78]:
cases_long = pd.melt(cases.reset_index(), id_vars='Date', var_name = 'Regions', value_name='Cases')

#Add the region code based on a dictionary.
dict_regions = {'Montréal':6, 'Montérégie':16, 'Laval':13, 'Estrie':5,
       'Mauricie - Centre-du-Québec':4, 'Lanaudière':14, 'Laurentides':15,
       'Capitale-Nationale':3, 'Chaudière-Appalaches':12, 'Outaouais':7,
       'Abitibi-Témiscamingue':8, 'Saguenay–Lac-Saint-Jean':2, 'Côte-Nord':9,
       'Gaspésie-Îles-de-la-Madeleine':11, 'Bas-Saint-Laurent':1, 'Eeyou Istchee':18,
       'Nord-du-Québec':10, 'Nunavik':17}
cases_long['Regions (code)'] = cases_long['Regions'].map(dict_regions)

In [113]:
#Get geographic shapefiles from MERN (https://mern.gouv.qc.ca/territoire/portrait/portrait-donnees-mille.jsp).
#Extract the zipfiles to BytesIO objects instead of saving them to a local directory. Then read them as a shapefile r.
#Finally, create a geopandas object from the shapefiles.
def geopandas_obj_from_zipped_shapefile(url):
    with urllib.request.urlopen(url) as dl_file:
        zipdata = BytesIO()
        zipdata.write(dl_file.read())
        with ZipFile(zipdata) as zip_ref:
            filenames = [y for y in sorted(zip_ref.namelist()) for ending in ['dbf', 'shp', 'shx'] if y.endswith(ending) and 'munic_polygone' in y] 
            dbf, shp, shx = [BytesIO(zip_ref.read(filename)) for filename in filenames]
            r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf, encoding="latin1")
    attributes, geometry = [], []
    field_names = [field[0] for field in r.fields[1:]]  
    for row in r.shapeRecords():  
        geometry.append(shape(row.shape.__geo_interface__))  
        attributes.append(dict(zip(field_names, row.record))) 
    #Create geopandas object.
    gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)
    return gdf
    
url = 'ftp://ftp.mrnf.gouv.qc.ca/public/dgig/produits/bdga1m/vectoriel/munic_SHP.zip'
gdf = geopandas_obj_from_zipped_shapefile(url)

In [114]:
#Now we want to read in our shapefile with Geopandas.
#For details of why we handle the shapefile object like this, see http://andrewgaidus.com/Reading_Zipped_Shapefiles/
#First, convert the attributes and geometry into a GeoJSON object.

#Clean the Quebec land to exclude all water.
muni_s_excl = [464,1375,1370]
gdf = gdf[(gdf['MUS_CO_DES']!='G') & (~gdf['MUS_CO_DES'].isna()) & (~gdf['MUNIC_S_'].isin(muni_s_excl))]
#Only keep columns we need
gdf = gdf[['MUS_CO_REG','MUS_NM_REG','geometry']]
#Rename columns.
gdf.columns = ['region_code', 'region_name', 'geometry']
#Change code to int
gdf['region_code'] = gdf['region_code'].astype('int')
#Dissolve municipalities to région administrative
gdf = gdf.dissolve(by='region_code')
#We also want to join Mauricie and Centre-du-Québec as one region since they report their COVID cases together.
gdf.loc[4] = pd.Series([gdf.loc[[4,17]].unary_union, 'Mauricie - Centre du Québec'],index=['geometry','region_name'])
gdf = gdf.drop(17,axis=0)

In [115]:
from bokeh.plotting import figure
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.palettes import brewer
from bokeh.layouts import column, row
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool, DateSlider, WheelZoomTool
#Define function that returns json_data for year selected by user.

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

def json_data(selected_date):
    date = selected_date
    df_date = cases_long[cases_long['Date'] == np.datetime64(date)]    
    merged = gdf.merge(df_date, left_on = 'region_code', right_on='Regions (code)',how='left')
    merged_json = json.loads(merged.to_json(default=json_serial))
    json_data = json.dumps(merged_json)
    return json_data

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data('2020-03-23'))

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 50, nan_color = '#d9d9d9')
#Add and format hover tool and zoom tool.
hover = HoverTool()
hover.tooltips = """
    <style>
        .bk-tooltip>div:not(:first-child) {display:none;}
    </style>

    <b>Région: </b> @Regions <br>
    <b>Cas: </b> @Cas
"""
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')
#Create figure object.
p = figure(title = 'Cas de COVID-19 (cumulatifs) par régions du Québec', plot_height = 600 , plot_width = 950, tools = ['wheel_zoom','pan'], toolbar_location = None)
p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Cas', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify layout
p.add_layout(color_bar, 'below')
# Define the callback function: update_plot
def update_plot(attr, old, new):
    #Convert from timestamp to date format that can then be transformed to iso by function json_serial.
    date = slider.value_as_date
    new_data = json_data(date)
    geosource.geojson = new_data
    p.title.text = 'Cas de COVID-19 (cumulatifs) par régions du Québec, %s' %date.isoformat()
    
# Make a slider object: slider li
slider = DateSlider(title="Date Range: ", start=date(2020, 3, 5), end=date.today(), value=date.today(), step=86400000)
slider.on_change('value', update_plot)
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = row(
    p,
    column(slider),
)

curdoc().add_root(layout)

##To generate the Bokeh graph, the easiest is to run the notebook from the command line (or anaconda prompt):  bokeh serve --show map_cases.ipynb